In [1]:
# Relative abundance calculations and stats for NanoCLUST amplicon data after post processing
# Jackson M. Tsuji, ILTS, Hokudai, 2022

In [2]:
import pandas as pd
from Bio.SeqIO.FastaIO import SimpleFastaParser

In [5]:
# Open NanoCLUST counts
cluster_counts_raw = pd.read_csv('../intermediate/cluster_counts_raw.tsv', sep='\t')
cluster_counts_raw['cluster_name'] = cluster_counts_raw['sample'] + '_cluster' + cluster_counts_raw['id'].apply(str)
cluster_counts_raw.head()

,sample,id,reads_in_cluster,used_for_consensus,reads_after_corr,draft_id,sciname,taxid,length,per_ident,cluster_name
0,20211112_barcode21,2,27946,100,40,a847cd6d-9263-4ef0-a5dc-60faa420bfe5 id=44,Ktedonobacter robiniae,2778365,1407,80.455,20211112_barcode21_cluster2
1,20211112_barcode21,0,92,100,38,2d5a9adf-599f-43b6-909d-6b1bdee75d2b id=29,Ktedonobacter robiniae,2778365,1407,80.455,20211112_barcode21_cluster0
2,20211112_barcode21,1,283,100,38,b86efd42-1105-4d66-a9fd-14fb4a341548 id=44,Ktedonobacter robiniae,2778365,1407,80.455,20211112_barcode21_cluster1
3,20220216_barcode08,7,14,100,13,381c9054-9f89-4515-b61f-81a607f794f9 id=7,Telmatobacter bradus,474953,1420,79.930,20220216_barcode08_cluster7
4,20220216_barcode08,8,38,100,37,1f8565a1-aab4-4bad-bdb1-d1dc30c6b056 id=31,Holophaga foetida,35839,1509,82.770,20220216_barcode08_cluster8


In [54]:
# Summarize total read count and # of clusters per sample
cluster_stats_raw = cluster_counts_raw\
  .groupby('sample').agg({'reads_in_cluster':sum,'id':'count'})\
  .reset_index()\
  .rename(columns={'id':'cluster_count'})
cluster_stats_raw['analysis_step'] = 'raw'

cluster_stats_raw.head()

,sample,reads_in_cluster,cluster_count,analysis_step
0,20211112_barcode21,28321,3,raw
1,20220216_barcode08,54451,13,raw
2,20220216_barcode11,51861,13,raw
3,20220216_barcode12,61995,6,raw
4,20220216_barcode13,67120,14,raw


In [14]:
# Get IDs of sequences that passed the CutAdapt filter

cluster_passing_cutadapt_names = []

with open('../intermediate/cluster_sequences_trimmed_complete.fasta') as handle:
    
    for cluster_name,sequence in SimpleFastaParser(handle):
        
        cluster_passing_cutadapt_names.append(cluster_name)

# Parse if reverse complemented
# TODO - this assumes that the only space is before 'rc' (the note left by CutAdapt on whether the sequence was reverse complemented)
#        I should only split off the last piece if it exactly matches ' rc'... and even this is a little dangerous if some of the sample names are odd.
clusters_passing_cutadapt = pd.Series(cluster_passing_cutadapt_names).str.split(' ', expand=True)
clusters_passing_cutadapt.columns = ['cluster_name', 'reverse_complemented']

# Bind on cluster stats
clusters_passing_cutadapt = clusters_passing_cutadapt.merge(cluster_counts_raw, on='cluster_name', 
                                                            how='left', validate='1:1')

clusters_passing_cutadapt.head()

,cluster_name,reverse_complemented,sample,id,reads_in_cluster,used_for_consensus,reads_after_corr,draft_id,sciname,taxid,length,per_ident
0,20211112_barcode21_cluster0,None,20211112_barcode21,0,92,100,38,2d5a9adf-599f-43b6-909d-6b1bdee75d2b id=29,Ktedonobacter robiniae,2778365,1407,80.455
1,20211112_barcode21_cluster1,None,20211112_barcode21,1,283,100,38,b86efd42-1105-4d66-a9fd-14fb4a341548 id=44,Ktedonobacter robiniae,2778365,1407,80.455
2,20211112_barcode21_cluster2,None,20211112_barcode21,2,27946,100,40,a847cd6d-9263-4ef0-a5dc-60faa420bfe5 id=44,Ktedonobacter robiniae,2778365,1407,80.455
3,20220216_barcode08_cluster0,None,20220216_barcode08,0,1579,100,40,b3b082f8-85d9-469d-831c-d904d37a4202 id=41,Ktedonobacter robiniae,2778365,1407,80.455
4,20220216_barcode08_cluster7,rc,20220216_barcode08,7,14,100,13,381c9054-9f89-4515-b61f-81a607f794f9 id=7,Telmatobacter bradus,474953,1420,79.930


In [53]:
# Summarize total read count and # of clusters per sample
# TODO - confirm 'count' does what I think it does
cluster_stats_passing_cutadapt = clusters_passing_cutadapt\
  .groupby('sample').agg({'reads_in_cluster':sum,'id':'count'})\
  .reset_index()\
  .rename(columns={'id':'cluster_count'})
cluster_stats_passing_cutadapt['analysis_step'] = 'cutadapt'

cluster_stats_passing_cutadapt.head()

,sample,reads_in_cluster,cluster_count,analysis_step
0,20211112_barcode21,28321,3,cutadapt
1,20220216_barcode08,54424,12,cutadapt
2,20220216_barcode11,51804,12,cutadapt
3,20220216_barcode12,61971,5,cutadapt
4,20220216_barcode13,66675,13,cutadapt


In [18]:
# Open OTU clustering file
otu_info = pd.read_csv('../intermediate/p99c99_cluster.tsv', sep='\t', header=None)
otu_info.columns = ['otu_id', 'cluster_name']
otu_info.head()

,otu_id,cluster_name
0,20211112_barcode21_cluster0,20211112_barcode21_cluster0
1,20211112_barcode21_cluster0,20211112_barcode21_cluster1
2,20211112_barcode21_cluster0,20211112_barcode21_cluster2
3,20211112_barcode21_cluster0,20220216_barcode08_cluster0
4,20211112_barcode21_cluster0,20220216_barcode11_cluster10


In [20]:
# Join to cutadapt passing table
otu_info = otu_info.merge(clusters_passing_cutadapt, on='cluster_name', how='outer', validate='1:1')
otu_info.head()

,otu_id,cluster_name,reverse_complemented,sample,id,reads_in_cluster,used_for_consensus,reads_after_corr,draft_id,sciname,taxid,length,per_ident
0,20211112_barcode21_cluster0,20211112_barcode21_cluster0,None,20211112_barcode21,0,92,100,38,2d5a9adf-599f-43b6-909d-6b1bdee75d2b id=29,Ktedonobacter robiniae,2778365,1407,80.455
1,20211112_barcode21_cluster0,20211112_barcode21_cluster1,None,20211112_barcode21,1,283,100,38,b86efd42-1105-4d66-a9fd-14fb4a341548 id=44,Ktedonobacter robiniae,2778365,1407,80.455
2,20211112_barcode21_cluster0,20211112_barcode21_cluster2,None,20211112_barcode21,2,27946,100,40,a847cd6d-9263-4ef0-a5dc-60faa420bfe5 id=44,Ktedonobacter robiniae,2778365,1407,80.455
3,20211112_barcode21_cluster0,20220216_barcode08_cluster0,None,20220216_barcode08,0,1579,100,40,b3b082f8-85d9-469d-831c-d904d37a4202 id=41,Ktedonobacter robiniae,2778365,1407,80.455
4,20211112_barcode21_cluster0,20220216_barcode11_cluster10,None,20220216_barcode11,10,123,100,38,b21f2a80-a0a7-44a2-b328-a86efd77978e id=95,Ktedonobacter robiniae,2778365,1407,80.455


In [55]:
# Summarize total read count and # of clusters per sample
cluster_stats_post_otu = otu_info\
  .groupby('sample').agg({'reads_in_cluster':sum,'otu_id':pd.Series.nunique})\
  .reset_index()\
  .rename(columns={'otu_id':'cluster_count'})
cluster_stats_post_otu['analysis_step'] = 'otu_clustering'

cluster_stats_post_otu.head()

,sample,reads_in_cluster,cluster_count,analysis_step
0,20211112_barcode21,28321,1,otu_clustering
1,20220216_barcode08,54424,9,otu_clustering
2,20220216_barcode11,51804,10,otu_clustering
3,20220216_barcode12,61971,4,otu_clustering
4,20220216_barcode13,66675,8,otu_clustering


In [26]:
# Collapse by OTU ID and by sample ID
otu_table_long = otu_info\
  .groupby(['sample','otu_id']).agg({'reads_in_cluster':sum})\
  .reset_index()

otu_table_long.head()

,sample,otu_id,reads_in_cluster
0,20211112_barcode21,20211112_barcode21_cluster0,28321
1,20220216_barcode08,20211112_barcode21_cluster0,1579
2,20220216_barcode08,20220216_barcode08_cluster1,52226
3,20220216_barcode08,20220216_barcode08_cluster12,145
4,20220216_barcode08,20220216_barcode08_cluster2,269


In [39]:
# Get IDs of sequences that passed the chimera filter

otu_passing_uchime_names = []
otu_passing_uchime_seqs = []

with open('../intermediate/chimera_filtered.fasta') as handle:
    
    for otu_id,sequence in SimpleFastaParser(handle):
        
        otu_passing_uchime_names.append(otu_id)
        otu_passing_uchime_seqs.append(sequence)

chimera_filtered_otus = pd.DataFrame({'otu_id':otu_passing_uchime_names, 'sequence':otu_passing_uchime_seqs})
# TODO - assumes the sequence doesn't end with ' rc' naurally
chimera_filtered_otus['otu_id'] = chimera_filtered_otus['otu_id'].str.replace(' rc$','',regex=True)

chimera_filtered_otus.head()

,otu_id,sequence
0,20211112_barcode21_cluster0,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...
1,20220216_barcode08_cluster1,AATGAACGCTGGCGGCGTGCCTAACACATGCAAGTCGGATGTGCCG...
2,20220216_barcode12_cluster5,ATTGAACGCTGGCGGCAGGCTTAACACATGCAAGTCGAGCGGGGAA...
3,20220216_barcode12_cluster0,GATGAACGCTGGCGGCGTGCTTAACACATGCAAGTCGAACGGTGAA...
4,20220216_barcode14_cluster0,AACGAACGCTGGCGGCATGCCTAACACATGCAAGTCGAACGATGCT...


In [40]:
# Bind on otu counts
otu_table_filtered = otu_table_long.merge(chimera_filtered_otus, on='otu_id', how='right', validate='m:1')

otu_table_filtered.head()

,sample,otu_id,reads_in_cluster,sequence
0,20211112_barcode21,20211112_barcode21_cluster0,28321,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...
1,20220216_barcode08,20211112_barcode21_cluster0,1579,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...
2,20220216_barcode11,20211112_barcode21_cluster0,13331,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...
3,20220216_barcode12,20211112_barcode21_cluster0,11,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...
4,20220216_barcode13,20211112_barcode21_cluster0,1842,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...


In [56]:
# Summarize total read count and # of clusters per sample
# TODO - confirm 'count' does what I think it does
cluster_stats_passing_uchime = otu_table_filtered\
  .groupby('sample').agg({'reads_in_cluster':sum,'otu_id':'count'})\
  .reset_index()\
  .rename(columns={'otu_id':'cluster_count'})
cluster_stats_passing_uchime['analysis_step'] = 'chimera_removal'

cluster_stats_passing_uchime.head()

,sample,reads_in_cluster,cluster_count,analysis_step
0,20211112_barcode21,28321,1,chimera_removal
1,20220216_barcode08,53805,2,chimera_removal
2,20220216_barcode11,50792,2,chimera_removal
3,20220216_barcode12,61971,4,chimera_removal
4,20220216_barcode13,65971,2,chimera_removal


In [49]:
# Make wide table - this will be the final table with count summary information
otu_table = otu_table_filtered.pivot(index='otu_id', columns='sample', values='reads_in_cluster')\
  .fillna(0)\
  .reset_index()

taxonomy = cluster_counts_raw[['cluster_name','sciname','taxid','per_ident']]\
  .rename(columns={'cluster_name':'otu_id'})

otu_table = otu_table.merge(taxonomy, on='otu_id', how='left', validate='1:1')\
  .merge(chimera_filtered_otus, on='otu_id', how='left', validate='1:1')

otu_table

,otu_id,20211112_barcode21,20220216_barcode08,20220216_barcode11,20220216_barcode12,20220216_barcode13,20220216_barcode14,20220527_barcode24,20220802_barcode22,20221116_barcode08,sciname,taxid,per_ident,sequence
0,20211112_barcode21_cluster0,28321.0,1579.0,13331.0,11.0,1842.0,0.0,34.0,0.0,0.0,Ktedonobacter robiniae,2778365,80.455,GATAAACGCTGGCGGCGTGCCTAATGCATGCAAGTCGAACGGGAGT...
1,20220216_barcode08_cluster1,0.0,52226.0,37461.0,61910.0,64129.0,63744.0,59029.0,75571.0,17398.0,Geothrix fermentans,44676,98.204,AATGAACGCTGGCGGCGTGCCTAACACATGCAAGTCGGATGTGCCG...
2,20220216_barcode12_cluster0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,Microbacterium testaceum,2033,99.249,GATGAACGCTGGCGGCGTGCTTAACACATGCAAGTCGAACGGTGAA...
3,20220216_barcode12_cluster5,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,Acinetobacter johnsonii,40214,99.467,ATTGAACGCTGGCGGCAGGCTTAACACATGCAAGTCGAGCGGGGAA...
4,20220216_barcode14_cluster0,0.0,0.0,0.0,0.0,0.0,43.0,0.0,0.0,0.0,Sphingomonas hankookensis,563996,99.441,AACGAACGCTGGCGGCATGCCTAACACATGCAAGTCGAACGATGCT...


In [50]:
# Save the count table
otu_table.to_csv('otu_table_counts.tsv', sep='\t', index=False)

In [63]:
# Summarize read count stats through the processing pipeline
cluster_stats_long = pd.concat([cluster_stats_raw, cluster_stats_passing_cutadapt,
                          cluster_stats_post_otu, cluster_stats_passing_uchime])
cluster_stats_long['analysis_step'] = pd.Categorical(cluster_stats_long['analysis_step'],
                                                    categories=['raw','cutadapt','otu_clustering',
                                                               'chimera_removal'], ordered=True)

cluster_stats_long.head()

,sample,reads_in_cluster,cluster_count,analysis_step
0,20211112_barcode21,28321,3,raw
1,20220216_barcode08,54451,13,raw
2,20220216_barcode11,51861,13,raw
3,20220216_barcode12,61995,6,raw
4,20220216_barcode13,67120,14,raw


In [66]:
cluster_stats = cluster_stats_long.pivot(index='sample', columns='analysis_step', 
                                         values=['reads_in_cluster','cluster_count'])\
  .reset_index()

cluster_stats

sample reads_in_cluster                          \
analysis_step                                  raw cutadapt otu_clustering   
0              20211112_barcode21            28321    28321          28321   
1              20220216_barcode08            54451    54424          54424   
2              20220216_barcode11            51861    51804          51804   
3              20220216_barcode12            61995    61971          61971   
4              20220216_barcode13            67120    66675          66675   
5              20220216_barcode14            63787    63787          63787   
6              20220527_barcode24            59087    59063          59063   
7              20220802_barcode22            75571    75571          75571   
8              20221116_barcode08            17411    17398          17398   

                              cluster_count                          \
analysis_step chimera_removal           raw cutadapt otu_clustering   
0                       28321             3        3              1   
1                       53805            13       12              9   
2                       50792            13       12             10   
3                       61971             6        5              4   
4                       65971            14       13              8   
5                       63787             2        2              2   
6                       59063             3        2              2   
7                       75571             2        2              1   
8                       17398             5        4              1   

                               
analysis_step chimera_removal  
0                           1  
1                           2  
2                           2  
3                           4  
4                           2  
5                           2  
6                           2  
7                           1  
8                           1

In [68]:
cluster_stats.to_csv('cluster_stats.tsv', sep='\t', index=False)